In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models import resnet18
import random
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

# Imports for Interface
import tkinter
from tkinter import *
from tkinter import ttk, messagebox
from PIL import ImageTk, Image
from tkinter import filedialog

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Constants and hyperparameters
num_classes = 2
image_size = 224
batch_size = 16

# Load the pre-trained ResNet model
model = resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
model = model.to(device)
model.load_state_dict(torch.load('../models/resnet_final_with_2class.pth'))
model.eval()

# Define the transformations for test data
transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the test dataset
test_dir = "../data/Test_2class"
test_dataset = ImageFolder(test_dir, transform=transform_test)
        
test_dataset = list(test_dataset)
test_dataset = random.sample(test_dataset, k=100)

# Create a DataLoader for managing the test data batches
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Iterate over the test data and make predictions
y_pred = []
y_true = []

for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    output = model(inputs)
    output = torch.argmax(torch.exp(output), dim=1).cpu().numpy()
    y_pred.extend(output)
    labels = labels.cpu().numpy()
    y_true.extend(labels)


def show_error(error_message):
    messagebox.showerror('Input Error', f'Error: {error_message}')


In [7]:

def Run(*args):
    try:
        
        b_s = int(batch_size_2.get())
        if not 250 <= b_s <= 500:
            show_error(f'Batch size have to be between 250 to 500')
            return
        s_s = int(sample_size_2.get())
        if not 5 <= s_s <= 100:
            show_error(f'Sample size have to be between 5 to 100')
            return
        n_r = int(nr_runs.get())
        if not 1 <= n_r <= 100:
            show_error(f'Number of runs have to be between 1 to 100')
            return
        u_a = float(user_aql.get())
        if not 0 < u_a < 100:
            show_error(f'Your AQL have to be between 1 to 100')
            return

        # Calculate the percentage of bad apples
        bach_size_32 = random.sample(y_pred, k=b_s)
        sample32 = random.sample(bach_size_32, k=s_s*n_r)
        Rot_Apple = sample32.count(1)
        Normal_Apple = sample32.count(0)
        perc_bad_apples = (Rot_Apple / len(sample32)) * 100

        # Classify the batch based on the percentage of bad apples
        if perc_bad_apples <= u_a:
            answer_1 = 'Class 1: Go to the supermarket'
        elif u_a < perc_bad_apples < 6.5:
            answer_1 = 'Class 2: Go to the applesauce factory'
        elif 6.5 <= perc_bad_apples < 15:
            answer_1 = 'Class 3: Go to the apple syrup factory'
        else:
            answer_1 = 'Class 4: Feed them to the pigs!'
            
        # Pie chart visualization for the types of apples
        labels = ['Normal', 'Rotten']
        sizes = [Normal_Apple, Rot_Apple]
        plt.pie(sizes, labels=labels, autopct='%1.1f%%')
        plt.axis('equal')  # Equal aspect ratio = drawn as a circle.
        plt.title('Distribution of Apple Types')
        plt.savefig('pics/pie_chart_2class.png')

        good_apple_percentage.set(f"{str(100 - perc_bad_apples)} %")
        test_accuracy.set(f"{str(99.89)} %")
        group_apple_category.set(str(answer_1))
        
    except ValueError:
        pass


def get_answer(*args):
    try:
        b_s = int(batch_size_2.get())
        s_s = int(sample_size_2.get())
        n_r = int(nr_runs.get())
        u_a = float(user_aql.get())
        
        # Calculate the percentage of bad apples
        bach_size_32 = random.sample(y_pred, k=b_s)
        sample32 = random.sample(bach_size_32, k=s_s*n_r)
        Rot_Apple = sample32.count(1)
        perc_bad_apples = (Rot_Apple / len(sample32)) * 100

        # Classify the batch based on the percentage of bad apples
        if perc_bad_apples <= u_a:
            answer_1 = 'Class 1: Go to the supermarket'
        elif u_a < perc_bad_apples < 6.5:
            answer_1 = 'Class 2: Go to the applesauce factory'
        elif 6.5 <= perc_bad_apples < 15:
            answer_1 = 'Class 3: Go to the apple syrup factory'
        else:
            answer_1 = 'Class 4: Feed them to the pigs!'
        
        
        # Load the pre-trained sentence transformer model
        model = SentenceTransformer('distilbert-base-nli-mean-tokens')
        
        # Define a dictionary of questions and answers
        qa_pairs = {
            "Can you tell me the total number of apples in the batch?": f"The total number of apples in the batch is {b_s}.",
            "What is the percentage of bad apples?": f"The percentage of bad apples is {perc_bad_apples:.2f}%.",
            "How many bad apples are there in an approved batch?": f"The number of bad apples in an approved batch is {Rot_Apple}.",
            "How many apples are categorized as blotch?": f"The number of apples categorized as blotch is {Rot_Apple}.",
            "What is the proportion of rotten apples?": f"The proportion of rotten apples is {Rot_Apple / len(sample32):.2f}.",
            "Can we use this apple batch to make apple mousse?": "Yes, we can use this apple batch to make apple mousse.",
            "Are there enough healthy apples to make apple sauce?": "To make apple sauce, we need to evaluate the number of healthy apples based on the desired quantity.",
            "Can we use this batch for the supermarket if the acceptance quality is increased by 1 percentage for the klasse 1?": f"If the acceptance quality is increased by 1 percentage for klasse 1, we can use this batch for the supermarket.",
            "Does the quality of the batch increase when the batch size is increased?": "The quality of the batch may or may not increase when the batch size is increased. It depends on various factors.",
            "What’s the average ratio between the healthy and unhealthy apples for different sample sizes?": "The average ratio between healthy and unhealthy apples can be calculated based on different sample sizes."
        }

        # Calculate sentence embeddings for the questions
        question_embeddings = model.encode(list(qa_pairs.keys()))
        
        def get_answer_2(user_query):
            query_embedding = model.encode([user_query]).flatten()  # Flatten the query_embedding
            similarities = [1 - cosine(query_embedding, q_emb) for q_emb in question_embeddings]
            most_similar_idx = similarities.index(max(similarities))
            return qa_pairs[list(qa_pairs.keys())[most_similar_idx]]

        # Chat bot loop
        
        user_query = str(chat_bot_box.get())
        if user_query.lower() == "exit":
            answer_2 = "Chat bot: Goodbye!"
        else:
            response = get_answer_2(user_query)
            answer_2 = f"Chat bot: {response}."
        
        chat_bot_answer.set(answer_2)
    except ValueError:
        pass
    
# Creating method for adding picture
def on_click():
    global my_img
    top = Toplevel()
    top.title('Apple Pie Chart')
    my_img = ImageTk.PhotoImage(Image.open('pics/pie_chart_2class.png'))
    Label(top, image=my_img).pack()
    
# Building interface
root = Tk()
root.title('Apple qualifier')

# Font properties
s = ttk.Style()
font_1 = ('Ariel Nova', 20)
s.configure('.', font = font_1)

mainframe = ttk.Frame(root, padding='3 3 12 12')
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)    

batch_size_2 = StringVar()
batch_size_entry = ttk.Entry(mainframe, width=10, font = font_1, textvariable=batch_size_2)
batch_size_entry.grid(column=2, row=1, sticky=(W, E))

sample_size_2 = StringVar()
sample_size_entry = ttk.Entry(mainframe, width=10, font = font_1, textvariable=sample_size_2)
sample_size_entry.grid(column=2, row=2, sticky=(W, E))

nr_runs = StringVar()
nr_runs_entry = ttk.Entry(mainframe, width=10, font = font_1, textvariable=nr_runs)
nr_runs_entry.grid(column=2, row=3, sticky=(W, E))

user_aql = StringVar()
user_aql_entry = ttk.Entry(mainframe, width=10, font = font_1, textvariable=user_aql)
user_aql_entry.grid(column=2, row=4, sticky=(W, E))

# Calculating Button (lower right)
good_apple_percentage = StringVar()
ttk.Label(mainframe, textvariable=good_apple_percentage).grid(column=2, row=6, sticky=(W, E))

test_accuracy = StringVar()
ttk.Label(mainframe, textvariable=test_accuracy).grid(column=2, row=7, sticky=(W, E))

group_apple_category = StringVar()
ttk.Label(mainframe, textvariable=group_apple_category).grid(column=2, row=8, sticky=(W, E))

# chat bot box
chat_bot_box = StringVar()
chat_bot_box_entry = ttk.Entry(mainframe, width=50, font = font_1, textvariable=chat_bot_box)
chat_bot_box_entry.grid(column=1, row=10, sticky=(W, E))

chat_bot_answer = StringVar()
ttk.Label(mainframe, textvariable=chat_bot_answer).grid(column=1, row=11, sticky=(W, E))

# Building interface Buttons
ttk.Button(mainframe, text='Info', command=on_click).grid(column=3, row=1, sticky=W)
ttk.Button(mainframe, text='Info', command=on_click).grid(column=3, row=2, sticky=W)
ttk.Button(mainframe, text='Info', command=on_click).grid(column=3, row=3, sticky=W)
ttk.Button(mainframe, text='Info', command=on_click).grid(column=3, row=4, sticky=W)
ttk.Button(mainframe, text='Run', command=Run).grid(column=3, row=5, sticky=W)
ttk.Button(mainframe, text='Chart', command=on_click).grid(column=3, row=6, sticky=W)


ttk.Button(mainframe, text='Run chat', command=get_answer).grid(column=3, row=10, sticky=W)
ttk.Label(mainframe, text='Please ask me a questions:').grid(column=1, row=9, sticky=W)

# Information Labels for Input Data Buttons
ttk.Label(mainframe, text='The batch size is:').grid(column=1, row=1, sticky=W)
ttk.Label(mainframe, text='The sample size is:').grid(column=1, row=2, sticky=W)
ttk.Label(mainframe, text='The number of sample run is:').grid(column=1, row=3, sticky=W)
ttk.Label(mainframe, text='The percentage of AQL is:').grid(column=1, row=4, sticky=W)
ttk.Label(mainframe, text='The percentage of good apples is:').grid(column=1, row=6, sticky=W)
ttk.Label(mainframe, text='Test accuracy is:').grid(column=1, row=7, sticky=W)
ttk.Label(mainframe, text='The apple group category is:').grid(column=1, row=8, sticky=W)

# Interface INPUT loop and OUTPUT
for child in mainframe.winfo_children(): 
    child.grid_configure(padx=30, pady=15)

batch_size_entry.focus()
sample_size_entry.focus()
nr_runs_entry.focus()
user_aql_entry.focus()
root.bind('<Return>', Run)

chat_bot_box_entry.focus()
root.bind('<Return>', get_answer)

root.mainloop()